Install the necessary libraries and packages for the prototype. We will use LangChain as the orchestration framework

In [17]:
!pwd
!pip install --upgrade pip

# Install required libraries
!python3 -m pip -q install redis
!pip install -U langchain gradio
!pip install -U langchain-core
!pip install -U langchain-community
!pip install -qU pypdf
!pip install -U redisvl
!pip install openai python-dotenv
!pip install -qU langchain-openai

/opt/app-root/src


In [ ]:
# Load Environment Variables, LLM Token, Embedding Model Token, and Redis DB password

In [18]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()



True

**Configure Redis Enterprise**

In [19]:
## Update the 'host' field with the correct Redis host URL
host = 'genai-db1-headless.redis-enterprise.svc.cluster.local'
port = 19838
password = os.getenv("REDIS_DB_PASS")
requirePass = True


In [20]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)

print(client.ping())
# Clear Redis database (optional)
client.flushdb()

REDIS_URL = f"redis://:{password}@{host}:{port}"
INDEX_NAME = f"idx_qna"

True


**Download sample PDF report and create knowledge base**

This PDF file will be used to get answers for the questions related to the contexts

In [21]:
!curl -L -o icici-bank-public-holidaylist.pdf "https://drive.google.com/uc?export=download&id=1-z04VOXN0gAYlhcPMlm2MhWLxkb2AxI2"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 57603  100 57603    0     0  22474      0  0:00:02  0:00:02 --:--:-- 45972


In [ ]:
# splitter

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

#from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader


file = "icici-bank-public-holidaylist.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=50, add_start_index=True
)

loader = PyPDFLoader(file)
documents = loader.load()

chunks = text_splitter.split_documents(documents)
#chunked_docs = [doc.page_content for doc in chunks]

In [ ]:
# optinal print the chunk

In [23]:
chunked_docs = [doc.page_content for doc in chunks]
print(chunks)

[Document(metadata={'source': 'icici-bank-public-holidaylist.pdf', 'page': 0, 'start_index': 0}, page_content="ICICIBankHolidayIndia\nHoliday Date&Day\nRepublicDay Sunday, 26January2025\nChhatrapati Shivaji Maharaj Jayanti Wednesday, 19February2025\nMahaShivaratri Wednesday, 26February2025\nHoli Friday, 14March2025\nUgadi/ Chetichand/Gudhi Padwa Sunday, 30March2025\nEidUl Fitr(Ramzan) Monday, 31March2025\nSriramaNavami Sunday, 6April 2025\nMahavirJanmaKalyanak Thursday, 10April 2025\nDr.B.R. Ambedkar'sBirthday/ Tamil NewYear Monday, 14April 2025\nGoodFriday Friday, 18April 2025\nMayDay/ MaharashtraDin Thursday, 1May2025\nBuddhaPurnima Monday, 12May2025\nEidul Azha(Bakrid) Saturday, 7June2025\nMuharram Sunday, 6July2025\nIndependenceDay Friday, 15August 2025\nVinayakaChaturthi Wednesday, 27August 2025\nMiladunNabi Friday, 5September2025\nMahatmaGandhi Jayanthi/ VijayaDasami Thursday, 2October2025\nDiwali Amavasya(Laxmi Pujan) Tuesday, 21October2025\nDiwali (Bali Pratipada)/ VikramSavant

In [ ]:
# Perform the embedding with nomic model storing the chunk into vector database

In [24]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain_openai import OpenAIEmbeddings

API_URL = "https://nomic-embed-text-v1-5-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443"
API_KEY = os.getenv("NOMIC_API_KEY")

embeddings = OpenAIEmbeddings(openai_api_base=API_URL, openai_api_key=API_KEY)


def get_vectordb() -> Redis:
    """Create the Redis vectordb."""
    # Load Redis with documents
    vectordb = Redis.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectordb


redis = get_vectordb()

# Include RAG

We're going to build a complete RAG pipeline from scratch incorporating the following components:

Standard retrieval and chat completion
Dense content representation to improve accuracy
Query re-writing to improve accuracy
Semantic caching to improve performance
Conversational session history to improve personalization

### Define Prompt template
PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [25]:
#@title Function to define prompt template

def create_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use only the following pieces of context to answer the question. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [Provide a detailed response based only on the context. Expand where possible.]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt


In [26]:
from langchain_community.llms import VLLMOpenAI

API_URL = "https://granite-3-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443"
API_KEY = os.getenv("LLM_API_KEY")


llm = VLLMOpenAI(
    openai_api_key=API_KEY,
    openai_api_base=API_URL + "/v1",
    model_name="granite-3-8b-instruct",
    model_kwargs={
        "stop": ["."],
        "max_tokens": 4096,  # Adjust this based on the model's supported limit
    },
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.5}),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": create_prompt()},
    #verbose=True
    )

/opt/app-root/lib64/python3.9/site-packages/IPython/core/interactiveshell.py:3490: UserWarning: Parameters {'max_tokens'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [27]:
qa.invoke('How many holidays are there in January 2025')['result']

' There is only one holiday in January 2025, which is RepublicDay on 26th January 2025'

In [28]:
qa.invoke('How many holidays are there in February 2025')['result']

'\n\nAnswer: There are two holidays in February 2025'

In [29]:
qa.invoke('Do we have holiday on Makar Sankranti in 2025?')['result']

' No, Makar Sankranti is not a holiday in 2025 according to the provided context'

In [ ]:
qa.invoke('How many total holidays are there in 2025')['result']

In [ ]:
qa.invoke('Are there any holidays which can help me to plan long weekend in February 2025?')['result']

In [ ]:
qa.invoke('what dates the holidays are in March 2025 ')['result']

In [ ]:
qa.invoke('What are some relevant items that are shopped online?')['result']

In [ ]:
import gradio as gr

def handle(query):
    response = qa.run(query)
    return response

iface = gr.Interface(fn=handle, inputs="text", outputs="text")
iface.launch(share=True)

In [ ]:
iface.close()